In [72]:
%matplotlib inline
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
import datetime
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import PredefinedSplit
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

C:\Users\Dong\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train=pd.DataFrame(pd.read_csv('fraud_train.csv',parse_dates=['date']))
#df_test=pd.DataFrame(pd.read_csv('fraud_test_a.csv'))

In [3]:
df_train_clean=df_train.loc[df_train['label']!=-1]
del df_train

In [4]:
cut_date_train=datetime.datetime(2017,10,20)
cut_date_test=datetime.datetime(2017,11,1)
day_distribute=df_train_clean['date'].value_counts()/df_train_clean.shape[0]

In [5]:

y=df_train_clean['label'].values
X=df_train_clean.drop(['id','label','date'],axis=1).values
x_train=X[df_train_clean.date<=cut_date_train]
x_test=X[df_train_clean.date>=cut_date_test]
y_train=y[df_train_clean.date<=cut_date_train]
y_test=y[df_train_clean.date>=cut_date_test]

#x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=69)


In [6]:
days=[(cut_date_train-date).days for date in df_train_clean.date]
days=np.array(days)
days_train=days[df_train_clean.date<=cut_date_train]

In [7]:
num_values=1000
category_feature=[]
for i in range(x_train.shape[1]):
    num=df_train_clean['f'+str(i+1)].unique().shape[0]
    if num <=num_values:
        category_feature.append(i)

In [8]:
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [62]:
estimator=lgb.LGBMClassifier(boosting_type='gbdt')

In [63]:
def atec_metric_cv(estimator,X,y):
    predict = estimator.predict(X)
    fpr,tpr,threshhold=roc_curve(y, predict)
    return 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1]

In [73]:
x_train_valid=np.concatenate((x_train,x_test),axis=0)
y_train_valid=np.concatenate((y_train,y_test),axis=0)
split=np.zeros(x_train_valid.shape[0])
split[:x_train.shape[0]]=-1
ps = PredefinedSplit(test_fold=split)

In [75]:
params_grid={
    
    'num_leaves': [31],
    'learning_rate': [0.1]
}
gbm_2=GridSearchCV(estimator,params_grid,scoring=atec_metric_cv,cv=ps)
gbm_2.fit(x_train_valid,y_train_valid,feature_name=feature_name, categorical_feature=category_feature)

C:\Users\Dong\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 1

GridSearchCV(cv=sklearn.cross_validation.PredefinedSplit(test_fold=[-1 -1 ...  0  0]),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'num_leaves': [31], 'learning_rate': [0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=<function atec_metric_cv at 0x0000029F21926AE8>, verbose=0)

In [76]:
gbm_2.best_score_

0.25988372093023254

In [70]:
x_train_valid.shape

(812882, 297)